In [1]:
"""
CONSTANTS FOR FAO PENMAN-MONTEITH CALCULATION
Region: Mersin / Tarsus (Agricultural Baseline)
"""

# ELEVATION (ALTITUDE)
# Tarsus district is selected as the reference point because it hosts the most 
# intensive agricultural production in Mersin and represents the characteristics 
# of the Çukurova basin. Using this official elevation is scientifically 
# optimal for calculating the atmospheric pressure (P) component of the formula.
elevation = 12  # meters (Official record for Tarsus)

# GEOGRAPHIC COORDINATES
# These coordinates represent the Tarsus district to ensure spatial consistency 
# with the elevation data. They serve as the official center point for 
# retrieving meteorological data from the NASA POWER database.
latitude = 36.91   # North
longitude = 34.89  # East

Metodolojik Yaklaşım: Evapotranspirasyon ($ET_0$) Kavramı ve FAO Penman-Monteith Hesaplama Mantığı1. 
Evapotranspirasyon Kavramının TanımıBu çalışmanın temelini oluşturan Evapotranspirasyon (ET), hidrolojik döngüde suyun yeryüzünden atmosfere transferini ifade eden iki temel fiziksel sürecin toplamıdır:
Evaporation (Buharlaşma): Suyun toprak yüzeyinden ve ıslak alanlardan fiziksel süreçlerle buharlaşarak atmosfere karışmasıdır.
Transpiration (Terleme): Bitkinin kökleriyle aldığı suyun, yaprak gözenekleri (stoma) yoluyla biyolojik bir süreç sonucunda buhar fazına geçerek kaybedilmesidir.
2. FAO-56 Penman-Monteith Yöntemi ve "Referans Bitki" YaklaşımıProjede kullanılan FAO Penman-Monteith metodolojisi, bu iki süreci (toprak buharlaşması ve bitki terlemesi) birbirinden ayırmadan, Potansiyel Evapotranspirasyon ($PET$ veya $ET_0$) adı verilen tek bir standart değer üretir. Model, matematiksel olarak "boyu 12 cm olan, yüzeyi tamamen gölgeleyen ve su kısıtı bulunmayan standart bir çim bitkisi (Referans Bitki)" varsayımı üzerinden çalışır.Bu yöntemle elde edilen $PET$ değeri, sadece biyolojik bir veri değil; sıcaklık, rüzgar, nem ve güneş radyasyonu parametrelerine bağlı olarak Mersin atmosferinin o ay ne kadar "su talep ettiğini" (Atmospheric Water Demand) gösteren bölgesel bir iklim göstergesidir. Dolayısıyla sonuç, hem topraktan hem de bitkiden kaynaklanan toplam su kaybı potansiyelini temsil eder.
3. Model Girdileri ve Yağış Değişkeninin KonumuPenman-Monteith formülasyonunda; sıcaklık, rüzgar hızı, buhar basıncı (nem) ve net radyasyon değişkenleri temel girdi olarak kullanılırken, "Toplam Yağış Miktarı" değişkeni bu formüle dahil edilmez. Bunun temel bilimsel gerekçesi şudur:Penman-Monteith formülü sistemin su giderini (atmosferin çekim gücünü) hesaplar.Yağış ise sistemin su gelirini (arz) oluşturur.Bu nedenle yağış verisi, PET hesaplandıktan sonraki aşamada devreye girer. Projenin kuraklık analizi (SPEI) aşamasında, hesaplanan su talebi ($PET$) ile su arzı ($Yağış$) arasındaki fark ($Su Bilançosu = Yağış - PET$) alınarak kuraklık şiddeti belirlenir.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [11]:
datasetraw = pd.read_csv(
    "06_Nasapower_Climate_Monthly_Mersin_Raw_for_Tmax_Tmin.csv", 
    sep=',',         # Ayırıcıyı kesin olarak VIRGÜL (,) olarak belirle
    header=10        # Sütun başlıkları dosyanın 23. satırında (indeks 22)
)

In [12]:
datasetraw

,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
0,T2M_MAX,1981,14.37,16.37,22.62,29.12,36.76,40.48,41.53,41.94,38.79,33.93,25.29,17.72,41.94
1,T2M_MAX,1982,17.54,16.49,20.17,26.74,33.95,40.52,40.13,41.94,40.12,30.72,25.84,18.34,41.94
2,T2M_MAX,1983,14.27,18.63,24.13,30.80,31.67,37.57,39.87,40.74,38.49,31.10,28.41,19.17,40.74
3,T2M_MAX,1984,16.29,18.18,22.54,26.93,33.68,39.56,41.26,39.65,37.78,36.46,21.36,17.49,41.26
4,T2M_MAX,1985,16.81,17.81,22.97,29.28,34.39,37.97,41.89,43.09,38.75,34.01,25.60,21.83,43.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,T2M_MIN,2021,-1.67,-1.45,2.55,5.81,14.02,17.36,22.62,23.19,15.49,13.91,7.93,-1.82,-1.82
86,T2M_MIN,2022,-2.55,0.83,-2.14,4.45,11.56,19.46,22.84,21.84,16.60,14.44,8.68,5.27,-2.55
87,T2M_MIN,2023,2.11,-0.79,2.84,4.74,9.49,17.41,21.48,24.46,20.48,16.23,3.32,5.31,-0.79
88,T2M_MIN,2024,2.06,2.54,5.40,11.82,11.27,17.47,23.48,25.37,17.71,11.43,2.92,1.79,1.79


In [13]:
datasetraw.columns

Index(['PARAMETER', 'YEAR', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ANN'],
      dtype='object')

In [14]:
# Tüm sütunların yeni isimleri, sırasına göre listelenmelidir
yeni_sutun_isimleri = ["Parameter","Year",'January','February','March','April','May','June','July','August','September','October','November','December','Annual Mean/Total Temperature']

datasetraw.columns = yeni_sutun_isimleri

parametre_degistirme_sozlugu = {
    'T2M_MAX': 'Tmax_at 2 Meters',
    'T2M_MIN': 'Tmin_at 2 Meters'
}

# 'PARAMETER' sütununu al, map() ile değerleri değiştir ve yeni değerleri geri ata.
datasetraw["Parameter"] = datasetraw["Parameter"].map(parametre_degistirme_sozlugu)
aylik_sutunlar = ['January','February','March','April','May','June','July','August','September','October','November','December','Annual Mean/Total Temperature']

for col in aylik_sutunlar:
    datasetraw[col] = datasetraw[col].replace(-999.00, datasetraw[col].median())


In [16]:
datasetraw

,Parameter,Year,January,February,March,April,May,June,July,August,September,October,November,December,Annual Mean/Total Temperature
0,Tmax_at 2 Meters,1981,14.37,16.37,22.62,29.12,36.76,40.48,41.53,41.94,38.79,33.93,25.29,17.72,41.940
1,Tmax_at 2 Meters,1982,17.54,16.49,20.17,26.74,33.95,40.52,40.13,41.94,40.12,30.72,25.84,18.34,41.940
2,Tmax_at 2 Meters,1983,14.27,18.63,24.13,30.80,31.67,37.57,39.87,40.74,38.49,31.10,28.41,19.17,40.740
3,Tmax_at 2 Meters,1984,16.29,18.18,22.54,26.93,33.68,39.56,41.26,39.65,37.78,36.46,21.36,17.49,41.260
4,Tmax_at 2 Meters,1985,16.81,17.81,22.97,29.28,34.39,37.97,41.89,43.09,38.75,34.01,25.60,21.83,43.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Tmin_at 2 Meters,2021,-1.67,-1.45,2.55,5.81,14.02,17.36,22.62,23.19,15.49,13.91,7.93,-1.82,-1.820
86,Tmin_at 2 Meters,2022,-2.55,0.83,-2.14,4.45,11.56,19.46,22.84,21.84,16.60,14.44,8.68,5.27,-2.550
87,Tmin_at 2 Meters,2023,2.11,-0.79,2.84,4.74,9.49,17.41,21.48,24.46,20.48,16.23,3.32,5.31,-0.790
88,Tmin_at 2 Meters,2024,2.06,2.54,5.40,11.82,11.27,17.47,23.48,25.37,17.71,11.43,2.92,1.79,1.790


In [18]:
# Aylık sütun isimleri
aylik_sutunlar = ['January','February','March','April','May','June','July','August','September','October','November','December']

# melt işlemi
datasetraw = datasetraw.melt(
    id_vars=["Parameter", "Year", "Annual Mean/Total Temperature"],  # Kimlik değişkenleri
    value_vars=aylik_sutunlar,  # Eritilecek değişkenler (aylık sütunlar)
    var_name="Month_Name",  # Yeni sütun adı
    value_name="Parameter_Value"  # Yeni değer sütun adı
)

# Sonuçta datasetraw veri çerçevesi, 5 sütun içerecek

In [19]:
datasetraw

,Parameter,Year,Annual Mean/Total Temperature,Month_Name,Parameter_Value
0,Tmax_at 2 Meters,1981,41.940,January,14.37
1,Tmax_at 2 Meters,1982,41.940,January,17.54
2,Tmax_at 2 Meters,1983,40.740,January,14.27
3,Tmax_at 2 Meters,1984,41.260,January,16.29
4,Tmax_at 2 Meters,1985,43.090,January,16.81
...,...,...,...,...,...
1075,Tmin_at 2 Meters,2021,-1.820,December,-1.82
1076,Tmin_at 2 Meters,2022,-2.550,December,5.27
1077,Tmin_at 2 Meters,2023,-0.790,December,5.31
1078,Tmin_at 2 Meters,2024,1.790,December,1.79


In [20]:
# Ayların metin karşılıkları ve sayısal karşılıkları
aylik_sutunlar = ['January','February','March','April','May','June','July','August','September','October','November','December']

# Ay adı - Ay sayısı eşleştirmesi
ay_map = {ay: i + 1 for i, ay in enumerate(aylik_sutunlar)}

# Yeni sütun oluşturma: Ay adlarını sayısal değere çevirir
datasetraw['Month_Number'] = datasetraw['Month_Name'].map(ay_map)

In [21]:
# data1_melted DataFrame'indeki Yıl ve Ay_Sayı sütunlarını kullanıyoruz
datasetraw['Date'] = (
    datasetraw['Year'].astype(str) + '-' + 
    datasetraw['Month_Number'].astype(str).str.zfill(2) # Ay sayısını iki haneli yapar (örn: '1' yerine '01')
)

In [22]:
datasetraw.drop(["Month_Number","Month_Name"],axis=1)

,Parameter,Year,Annual Mean/Total Temperature,Parameter_Value,Date
0,Tmax_at 2 Meters,1981,41.940,14.37,1981-01
1,Tmax_at 2 Meters,1982,41.940,17.54,1982-01
2,Tmax_at 2 Meters,1983,40.740,14.27,1983-01
3,Tmax_at 2 Meters,1984,41.260,16.29,1984-01
4,Tmax_at 2 Meters,1985,43.090,16.81,1985-01
...,...,...,...,...,...
1075,Tmin_at 2 Meters,2021,-1.820,-1.82,2021-12
1076,Tmin_at 2 Meters,2022,-2.550,5.27,2022-12
1077,Tmin_at 2 Meters,2023,-0.790,5.31,2023-12
1078,Tmin_at 2 Meters,2024,1.790,1.79,2024-12


In [24]:
datasetraw1= datasetraw.pivot_table(
    index="Date",      # Yeni Tablonun Satırları
    columns='Parameter',           # Yeni Tablonun Sütun Başlıkları
    values="Parameter_Value"                # Hücrelere Yerleşecek Veriler
)
datasetraw1.columns.name = None

In [25]:
datasetraw1

,Tmax_at 2 Meters,Tmin_at 2 Meters
Date,,
1981-01,14.37,0.40
1981-02,16.37,1.25
1981-03,22.62,3.47
1981-04,29.12,2.41
1981-05,36.76,7.65
...,...,...
2025-08,47.23,24.61
2025-09,39.93,19.42
2025-10,33.35,12.09


In [26]:
# Sütun adlarını ve birimlerini tanımlayan sözlük
yeni_sutun_adlari = {
    'Tmax_at 2 Meters': 'Tmax_at 2 Meters(°C)',
    'Tmin_at 2 Meters': 'Tmin_at 2 Meters(°C)'}

# DataFrame'deki sütun adlarını yeniden adlandırma
datasetraw1 = datasetraw1.rename(columns=yeni_sutun_adlari)

In [27]:
import matplotlib.dates as mdates

In [28]:
datasetraw1.columns = datasetraw1.columns.str.strip()
datasetraw1

,Tmax_at 2 Meters(°C),Tmin_at 2 Meters(°C)
Date,,
1981-01,14.37,0.40
1981-02,16.37,1.25
1981-03,22.62,3.47
1981-04,29.12,2.41
1981-05,36.76,7.65
...,...,...
2025-08,47.23,24.61
2025-09,39.93,19.42
2025-10,33.35,12.09


In [30]:
if 'Date' not in datasetraw1.columns and datasetraw1.index.name == 'Date':
    datasetraw1 = datasetraw1.reset_index()

# Tarih formatını datetime yap
datasetraw1['Date'] = pd.to_datetime(datasetraw1['Date'])

In [31]:
datasetraw1=datasetraw1.sort_values(by='Date')

In [32]:
datasetraw1

,Date,Tmax_at 2 Meters(°C),Tmin_at 2 Meters(°C)
0,1981-01-01,14.37,0.40
1,1981-02-01,16.37,1.25
2,1981-03-01,22.62,3.47
3,1981-04-01,29.12,2.41
4,1981-05-01,36.76,7.65
...,...,...,...
535,2025-08-01,47.23,24.61
536,2025-09-01,39.93,19.42
537,2025-10-01,33.35,12.09
538,2025-11-01,10.84,10.84


In [33]:
datasetraw1.to_csv("07_Nasapower_Climate_Monthly_Mersin_for_Tmax_Tmin_Preprocessed.csv",index=False)

In [34]:
datasetorj=pd.read_csv("05_Nasapower_Climate_Monthly_Mersin_Finalversion.csv")

In [36]:
datasetorj['Date'] = pd.to_datetime(datasetorj['Date'])

In [37]:
 datasetorj = datasetorj.reset_index(drop=True)

In [39]:
# Hata veren sütunları siler ve sonucu tekrar datasetorj değişkenine eşitler
#datasetorj = datasetorj.drop(columns=["index", "Unnamed: 0"]) if we need
datasetorj

,Date,2m Air Temperature (C),2m Relative Humidity (%),2m Wind Speed (m/s),Global Solar Radiation (MJ/m2/day),Total Precipitation (mm)
0,1981-01-01,1.32,82.24,3.50,7.75,174.17
1,1981-02-01,1.11,83.43,2.46,11.20,88.69
2,1981-03-01,5.94,76.62,2.54,15.57,24.21
3,1981-04-01,8.70,66.20,2.03,18.91,26.42
4,1981-05-01,12.27,61.53,2.15,22.75,26.41
...,...,...,...,...,...,...
535,2025-08-01,26.03,31.23,2.09,22.98,1.90
536,2025-09-01,19.84,39.16,2.13,19.22,3.19
537,2025-10-01,14.06,14.06,14.06,14.06,14.06
538,2025-11-01,9.39,9.39,9.39,9.39,9.39


In [40]:
yeni_veri_seti = pd.merge(datasetorj, datasetraw1, on='Date', how='left')

In [41]:
yeni_veri_seti

,Date,2m Air Temperature (C),2m Relative Humidity (%),2m Wind Speed (m/s),Global Solar Radiation (MJ/m2/day),Total Precipitation (mm),Tmax_at 2 Meters(°C),Tmin_at 2 Meters(°C)
0,1981-01-01,1.32,82.24,3.50,7.75,174.17,14.37,0.40
1,1981-02-01,1.11,83.43,2.46,11.20,88.69,16.37,1.25
2,1981-03-01,5.94,76.62,2.54,15.57,24.21,22.62,3.47
3,1981-04-01,8.70,66.20,2.03,18.91,26.42,29.12,2.41
4,1981-05-01,12.27,61.53,2.15,22.75,26.41,36.76,7.65
...,...,...,...,...,...,...,...,...
535,2025-08-01,26.03,31.23,2.09,22.98,1.90,47.23,24.61
536,2025-09-01,19.84,39.16,2.13,19.22,3.19,39.93,19.42
537,2025-10-01,14.06,14.06,14.06,14.06,14.06,33.35,12.09
538,2025-11-01,9.39,9.39,9.39,9.39,9.39,10.84,10.84


In [42]:
# Sütun adlarını ve birimlerini tanımlayan sözlük
yeni_sutun_adi = {
    '2m Air Temperature (C)': '2m Air Temperature (°C)'}

# DataFrame'deki sütun adlarını yeniden adlandırma
yeni_veri_seti = yeni_veri_seti.rename(columns=yeni_sutun_adi)

In [43]:
yeni_veri_seti.to_csv("08_Nasapower_Climate_Monthly_Mersin_ExactFinalVersion.csv",index=False)

In [44]:
yeni_veri_seti

,Date,2m Air Temperature (°C),2m Relative Humidity (%),2m Wind Speed (m/s),Global Solar Radiation (MJ/m2/day),Total Precipitation (mm),Tmax_at 2 Meters(°C),Tmin_at 2 Meters(°C)
0,1981-01-01,1.32,82.24,3.50,7.75,174.17,14.37,0.40
1,1981-02-01,1.11,83.43,2.46,11.20,88.69,16.37,1.25
2,1981-03-01,5.94,76.62,2.54,15.57,24.21,22.62,3.47
3,1981-04-01,8.70,66.20,2.03,18.91,26.42,29.12,2.41
4,1981-05-01,12.27,61.53,2.15,22.75,26.41,36.76,7.65
...,...,...,...,...,...,...,...,...
535,2025-08-01,26.03,31.23,2.09,22.98,1.90,47.23,24.61
536,2025-09-01,19.84,39.16,2.13,19.22,3.19,39.93,19.42
537,2025-10-01,14.06,14.06,14.06,14.06,14.06,33.35,12.09
538,2025-11-01,9.39,9.39,9.39,9.39,9.39,10.84,10.84
